# Query Expansion

0) Just some imports

In [484]:
import re
import math
import numpy as np
import common as cm

# 1) Simple search engine

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [485]:
class Dictionary:
    def __init__(self):     
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        self.corM = [] ### a correlation matrix

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeIDFs(self, documents):
        self.idfs = []
        for t in self.terms:
            counter = 0
            for d in documents:
                counter += d.tokens.count(t)
            self.idfs.append(np.log2(len(documents)/counter))
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeCorM(self, documents):
        self.corM = []
        for d1 in documents:
            row = []
            d1Set = set(d1.tokens)
            for d2 in documents:
                d2Set = set(d2.tokens)
                z = d1Set.intersection(d2Set)
                y = d1Set.union(d2Set)
                if d1 == d2:
                    row.append(-1.0)
                else:
                    row.append(len(z)/len(y))
            self.corM.append(row)
        

### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [486]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [487]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [488]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = [] # Bag Of Words (BOW - number of term occurences)
        self.tf = [] # TF representation
        self.tf_idf = [] # TF-IDF representation

    ### TODO - complete this method; it should compute a BOW representation
    def computeBOW_Representation(self):
        self.bow = []
        for t in self.dictionary.terms:
            self.bow.append(self.tokens.count(t))
    
    ### TODO - complete this method; it should compute a TF representation
    def computeTF_Representation(self):
        self.tf = []
        m = max(self.bow)
        for n in self.bow:
            self.tf.append(n/m) 
    
    ### TODO - complete this method; it should compute a TFxIDF representation 
    ### (important: it should not be run before dictionary.idfs are not computed!)
    def computeTF_IDF_Representation(self):
        self.tf_idf = [] 
        for i, idf in enumerate(self.dictionary.idfs):
            self.tf_idf.append(idf*self.tf[i])
    
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]


1.4) Compute IDFs here

In [489]:
### TODO COMPUTE IDFS HERE (FINISH THE PROPER METHOD OF THE DICTIONARY CLASS - DO NOT FORGET TO RE-RUN THE CELL)
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms))]
res.sort(key = lambda x: x[1])
# LEAST COMMON WORDS - HIGH IDF
print(res[-5:])
# MOST COMMON WORDS - LOW IDF
print(res[:5])

[['working', 6.459431618637297], ['www', 6.459431618637297], ['york', 6.459431618637297], ['young', 6.459431618637297], ['zdm', 6.459431618637297]]
[['learning', -1.3797721694596468], ['machine', -1.3479233034203069], ['the', -0.14048122354983042], ['and', 0.11958161575267245], ['for', 1.1375035237499351]]


1.5) Compute the document representations (for each document run computeRepresentations())

In [490]:
for d in documents: d.computeRepresentations()
### SOME DEBUG (you should see some 4s - which terms are these?)
print(documents[0].bow)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [491]:
### TODO 
def getSimilarity(v1, v2):
    sumV1 = 0
    sumV2 = 0
    nominator = 0
    for tf1, tf2 in zip(v1.tf_idf, v2.tf_idf):
        sumV1 += tf1 ** 2
        sumV2 += tf2 ** 2
        nominator += tf1 * tf2
    denominator = math.sqrt(sumV1 * sumV2)  
    return nominator/denominator

1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [492]:
query = "machine learning"
# query = "academic research"
# query = "international conference"
# query = "international conference washington"

In [493]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ranks = [[d, getSimilarity(d, qd)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 0.5962240026792148 | DOC ID = 63
AI / Machine Learning Resources
 AI / Machine Learning Resources. General Machine Learning. The Journal
 of Machine Learning. MLnet Machine Learning Archive at GMD. The ... 


RANK = 2 WITH SIMILARITY = 0.5112284831166727 | DOC ID = 77
Machine Learning
 Machine Learning. Machine Learning Home Page (Editor) Machine Learning Home
 Page (Publisher) Machine Learning Online by Kluwer Academic Publishers: ... 


RANK = 3 WITH SIMILARITY = 0.4592388053282037 | DOC ID = 34
Machine Learning
 Machine Learning. Related Sites. Machine Learning Resources courtesy
 of David Aha A Machine Learning Tutorial a good overview of the ... 


RANK = 4 WITH SIMILARITY = 0.4365976340007724 | DOC ID = 81
Oxford University Machine Learning Group
 Machine Learning at the Computing Laboratory. ... Logic Programming and
 Learning and Intelligent Systems. Other Machine Learning Groups. ... 


RANK = 5 WITH SIMILARITY = 0.4056657113272317 | DOC ID = 36
Mach

# 2) Query expansion

## 2.1) Correlation matrix

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [494]:
### TODO - COMPLETE THE computeCorM METHOD (see one of the first cells)
dictionary.computeCorM(documents)
print(dictionary.corM)

428571428572, 0.10714285714285714, 0.14814814814814814, 0.06896551724137931, 0.05555555555555555, 0.16, 0.1111111111111111, 0.14814814814814814, 0.08, 0.16666666666666666, 0.0967741935483871, 0.13043478260869565, 0.09090909090909091, 0.1111111111111111, 0.14705882352941177, 0.10344827586206896, 0.07142857142857142, 0.09090909090909091, 0.07692307692307693, 0.13043478260869565, 0.07692307692307693, 0.0625, 0.12, 0.16, 0.20833333333333334, 0.08333333333333333, 0.08, 0.06666666666666667, 0.17391304347826086, 0.17857142857142858, 0.09523809523809523, 0.16, 0.1111111111111111, 0.07692307692307693, 0.10344827586206896, 0.10526315789473684, 0.16666666666666666, 0.15384615384615385, 0.125, 0.08571428571428572, 0.18181818181818182, 0.18518518518518517, 0.1282051282051282, 0.20588235294117646, 0.12, 0.125, 0.1, 0.13043478260869565, 0.08333333333333333, 0.17391304347826086, 0.25, 0.1111111111111111, 0.09090909090909091, 0.07407407407407407, 0.09523809523809523, 0.058823529411764705, 0.08333333333

2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [495]:
query = "machine"
# query = "algorithm"
# query = "learning"
# query = "conference"
# query = "research"
# query = "concept"

def suggestKeywords(query, dictionary):
    ### TODO
    matrix = []
    for i in range(len(dictionary.terms)):
        row = []
        suma = 0
        for d in documents:
            suma += d.bow[i]**2
            row.append(d.bow[i])
        row = [i / math.sqrt(suma) for i in row]
        matrix.append(row)
    matrix = np.asarray(matrix)
    matrix = matrix@matrix.T
    query_i = dictionary.terms.index(query)
    for i in range(len(matrix)):
        matrix[i, i] *= -1.0
    m = max(matrix[query_i])
    cor = 0
    for i in range(len(matrix[query_i])):
        if matrix[query_i, i] == m:
            cor = i
    
    print(dictionary.terms[cor])
        
suggestKeywords(query, dictionary)

learning


# 2.2) Rocchio algorithm

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) Firstly, run the below code. Observe the results. Assume that we do not like the first and the second result (Docs 63 and 77). However, assume that docs 29 and 36 are satisfactory. Now, modfify the method. It should alter the query vector, according to Rocchio algorithm. Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77). Check the results for different values of alpha, beta, and gamma coefficients. 

In [496]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0.5,
                          beta = 0.3,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ##### TODO: MODIFY qd.tf_idf HERE
    A = [alpha * i for i in qd.tf_idf]
    suma = np.zeros(len(documents[0].tf_idf))
    for i in rel_docs:
        for b in range(len(suma)):
            suma[b] += documents[i].tf_idf[b]

    qd.tf_idf = suma
    #####
    ranks = [[d, getSimilarity(d, qd)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

RANK = 1 WITH SIMILARITY = 0.8256488742012702 | DOC ID = 29
Machine Learning
 6.858/18.428: Machine Learning. ... This course deals with the following topics:
 Formal models of machine learning; Learning concepts from examples; ... 


RANK = 2 WITH SIMILARITY = 0.733111425632266 | DOC ID = 36
Machine Learning Group
 [ Bristol CS | Index | Research | ML group | Student projects ] Machine
 Learning Research Group. Overview. The Machine Learning Research ... 
 Description: Research on higher-order concept learning, inductive logic programming, multi-agent learning systems,...


RANK = 3 WITH SIMILARITY = 0.337336032623368 | DOC ID = 81
Oxford University Machine Learning Group
 Machine Learning at the Computing Laboratory. ... Logic Programming and
 Learning and Intelligent Systems. Other Machine Learning Groups. ... 


RANK = 4 WITH SIMILARITY = 0.3296147093692282 | DOC ID = 13
UTCS Machine Learning Research Group
 Machine learning is the study of adaptive computational systems that
 impr

# 2.3) WordNet

2.3.1) Installation

http://www.nltk.org/install.html

import nltk 

nltk.download()

https://www.nltk.org/data.html

In [497]:
from nltk.corpus import wordnet as wn
import nltk

ntlk.download()

ModuleNotFoundError: No module named 'nltk'

Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [498]:
wn.synsets('machine')

NameError: name 'wn' is not defined

2.3.3) Display all definitions (.definition()) for synsets (machine)

In [499]:
#TODO
print(wn.synsets('machine').definition())

NameError: name 'wn' is not defined

2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [500]:
#TODO
print(wn.synets('machine').hypernym())

NameError: name 'wn' is not defined

See: http://www.nltk.org/howto/wordnet.html
for more examples